In [1]:
# !pip install graphviz

# Modello HCPN per Smart Hospital


In [2]:
from pathlib import Path

import random
import matplotlib.pyplot as plt
from collections import defaultdict
import os


os.makedirs("plots", exist_ok=True)

# Set seed for reproducibility
random.seed(42)

# Define places in the system

In [3]:
places = [
    "ECG_gen", "O2_gen", "Vent_gen", "Diag_gen",
    "EdgeA_buff", "EdgeB_buff", "EdgeA_proc", "EdgeB_proc",
    "SecA_monitor", "SecB_monitor", "Central_proc",
    "ICU_disp", "ER_disp"
]


# Define transitions with probabilities (realistic, literature-based)

In [4]:

transitions = {
    "ECG_gen -> EdgeA_buff": 0.6,
    "O2_gen -> EdgeA_buff": 0.6,
    "Vent_gen -> EdgeB_buff": 0.4,
    "Diag_gen -> EdgeB_buff": 0.4,
    "EdgeA_buff -> EdgeA_proc": 0.7,
    "EdgeB_buff -> EdgeB_proc": 0.6,
    "EdgeA_proc -> SecA_monitor": 0.65,
    "EdgeB_proc -> SecB_monitor": 0.55,
    "SecA_monitor -> Central_proc": 0.75,
    "SecB_monitor -> Central_proc": 0.65,
    "Central_proc -> ICU_disp": 0.5,
    "Central_proc -> ER_disp": 0.5
}



# Parameters

In [5]:

num_simulations = 1000
num_steps = 100



# Initialize results


In [6]:
place_token_history = defaultdict(list)
transition_throughput = defaultdict(int)
latency_records = []


# Run simulations

In [7]:
for sim in range(num_simulations):
    # Initialize tokens for each simulation
    markings = defaultdict(list)
    token_birth = {}

    for step in range(num_steps):
        # 1. Generate new tokens at source places
        for dev in ["ECG", "O2", "Vent", "Diag"]:
            if random.random() < 0.4:
                token = (sim, step, random.randint(1, 50))
                place = f"{dev}_gen"
                markings[place].append(token)
                token_birth[token] = step

        # 2. Transition firings
        for transition, prob in transitions.items():
            src, dst = transition.split(" -> ")
            if markings[src] and random.random() < prob:
                token = markings[src].pop(0)
                markings[dst].append(token)
                transition_throughput[transition] += 1
                # If transition to final place, compute latency
                if dst in ["ICU_disp", "ER_disp"]:
                    birth = token_birth.get(token)
                    if birth is not None:
                        latency = step - birth
                        latency_records.append(latency)

        # 3. Record token count in each place
        for place in places:
            place_token_history[place].append(len(markings[place]))


In [8]:

# Save all plots
Path("plots").mkdir(exist_ok=True)

# 1. Mean token plot
avg_tokens = {p: sum(ts)/len(ts) for p, ts in place_token_history.items()}
plt.figure(figsize=(14, 6))
plt.bar(avg_tokens.keys(), avg_tokens.values(), color="steelblue")
plt.xticks(rotation=45, ha="right")
plt.title("Mean Number of Tokens per Place (100 Simulations)")
plt.ylabel("Average Token Count")
plt.tight_layout()
plt.savefig("plots/mean_tokens.png", dpi=300)
plt.close()

# 2. Throughput
throughput_sorted = dict(sorted(transition_throughput.items(), key=lambda x: x[1], reverse=True))
plt.figure(figsize=(14, 6))
plt.bar(throughput_sorted.keys(), throughput_sorted.values(), color="tomato")
plt.xticks(rotation=45, ha="right")
plt.title("Transition Throughput over 100 Simulations")
plt.ylabel("Fired Count")
plt.tight_layout()
plt.savefig("plots/throughput.png", dpi=300)
plt.close()

# 3. Latency
plt.figure(figsize=(10, 5))
plt.hist(latency_records, bins=25, color="darkgreen", edgecolor="black", alpha=0.8)
plt.title("Latency Distribution from Source to ICU/ER")
plt.xlabel("Latency (Steps)")
plt.ylabel("Frequency")
plt.tight_layout()
plt.savefig("plots/latency_hist.png", dpi=300)
plt.close()